# Pickhardt Payments Example

Example code demonstrating how to use the `pickhardtpayments` package in python. You need to install the library via `pip install pickhardtpayments` or you can download the full source code at https://ln.rene-pickhardt.de or a copy from github at: https://www.github.com/renepickhardt/pickhardtpayments

Of course you can use the classes int the library to create your own async payment loop or you could exchange the Oracle to talk to the actual Lightning network by wrapping against your favourite node implementation and exposing the the `send_onion` call. 

This example assumes a randomly generated Oracle to conduct payments in a simulated way. For this you will need an actual channelgraph which you can get for example with `lightning-cli listchannels > listchannels20220412.json`

## Acknowledgements & Funding
This work is funded via various sources including NTNU & BitMEX as well as many generous donors via https://donate.ln.rene-pickhardt.de or https://www.patreon.com/renepickhardt Feel free to go to my website at https://ln.rene-pickhardt.de to learn how I have been contributing to the open source community and why it is important to have independent open source contributors. In case you also wish to support me I will be very grateful

In [1]:
sys.path.append('..')
from pickhardtpayments.ChannelGraph import ChannelGraph
from pickhardtpayments.UncertaintyNetwork import UncertaintyNetwork
from pickhardtpayments.OracleLightningNetwork import OracleLightningNetwork
from pickhardtpayments.SyncSimulatedPaymentSession import SyncSimulatedPaymentSession

In [2]:
#Rene Pickhardt's public node key
RENE = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
#Carsten Otto's public node key
C_OTTO = "027ce055380348d7812d2ae7745701c9f93e70c1adeb2657f053f91df4f2843c71"

#we first need to import the chanenl graph from c-lightning jsondump
#you can get your own data set via:
# $: lightning-cli listchannels > listchannels20220412.json
# alternatively you can go to https://ln.rene-pickhardt.de to find a data dump
channel_graph = ChannelGraph("listchannels20220412.json")

#we now create ourself an Oracle. This is Simulated Network that assumes unformly distribution 
#of the liquidity for the channels on the `channel_graph`. Of course one could create ones own 
#oracle (for example a wrapper to an existing lightning network node / implementation)
oracle_lightning_network = OracleLightningNetwork(channel_graph)

In [3]:
# Since we randomly generated our Oracle but also since we control it we can compute
# The maximum possible amout that can be payed between two nodes
maximum_payable_amount =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,1000)
print(maximum_payable_amount, "sats would be possible on this oracle to deliver if including 1 sat basefee channels")


133487843 sats would be possible on this oracle to deliver if including 1 sat basefee channels


In [4]:
#Of course we want to restrict ourselves to the zeroBaseFee part of the network.
#Therefor we compute the theoretical maximum payable amount for that subgraph
maximum_payable_amount =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,0)
print(maximum_payable_amount, "sats possible on this oracle on the zeroBaseFeeGraph")

71395508 sats possible on this oracle on the zeroBaseFeeGraph


In [5]:
# We chose an amount that is 50% of half the theoretic maximum to demonstrate the the
# minimum cost flow solver with Bayesian updates on the Uncertainty Network finds the
# liquidity rather quickly
tested_amount = int(maximum_payable_amount/2)

# From the channel graph we can derrive our initial Uncertainty Network which is the main data structure
# that we maintain in order to deliver sats from one node to another
uncertainty_network = UncertaintyNetwork(channel_graph)

#we create ourselves a payment session which in this case operates by sending out the onions
#sequentially 
payment_session = SyncSimulatedPaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False)

#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=0,base=0)

Round number:  1
Try to deliver 35697754 satoshi:

Statistics about 10 candidate onions:

successful attempts:
--------------------
 p =  55.96% amt:   2577528 sats  hops: 3 ppm:  1896
 p =  70.99% amt:   1000000 sats  hops: 4 ppm:  2060
 p =  33.24% amt:   3354000 sats  hops: 5 ppm:  2551

failed attempts:
----------------
 p =  68.30% amt:   2020000 sats  hops: 3 ppm:  1974 
 p =  10.70% amt:  13421772 sats  hops: 3 ppm:  2036 
 p =  67.46% amt:   1000000 sats  hops: 4 ppm:  2187 
 p =  48.71% amt:   3355443 sats  hops: 4 ppm:  2496 
 p =  45.21% amt:   3000000 sats  hops: 5 ppm:  2580 
 p =  44.24% amt:    969011 sats  hops: 5 ppm:  2057 
 p =  22.97% amt:   5000000 sats  hops: 6 ppm:  5318 

Attempt Summary:

Tried to deliver   35697754 sats
expected to deliver   11326524 sats 	(31.73%)
actually deliverd    6931528 sats 	(19.42%)
deviation: 0.61
planned_fee: 93726.367 sat
paid fees: 15508.976 sat
Runtime of flow computation: 0.21 sec 


Round number:  2
Try to deliver 28766226 sato

## Optimizing for Fees

controlling mu we can decide how much we wish to focuse on lower fees. However we will see that it will be much harder to deliver the same amount in the sense that we need to send out more onions and also have more failed attampts. Consiquantly we expect to need more time.

In [6]:
payment_session = SyncSimulatedPaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False,
                                 use_negative_circle_solver=False)
#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=100,base=0)

Round number:  1
Try to deliver 35697754 satoshi:

Statistics about 22 candidate onions:

successful attempts:
--------------------
 p =  74.13% amt:    220000 sats  hops: 2 ppm:  1339
 p =  46.54% amt:   2000000 sats  hops: 3 ppm:  1352
 p =  81.28% amt:      7215 sats  hops: 4 ppm:  1417
 p =  44.38% amt:    260000 sats  hops: 4 ppm:  1377
 p =   0.00% amt:   2000000 sats  hops: 4 ppm:  1379
 p =   5.14% amt:   3528865 sats  hops: 5 ppm:  1352
 p =   3.25% amt:   4328539 sats  hops: 5 ppm:  1361
 p =   1.19% amt:    142596 sats  hops: 6 ppm:  1361

failed attempts:
----------------
 p =  78.46% amt:    192000 sats  hops: 2 ppm:  1342 
 p =  36.00% amt:   2000000 sats  hops: 2 ppm:  1360 
 p =  59.84% amt:    640000 sats  hops: 3 ppm:  1416 
 p =  10.94% amt:   6071135 sats  hops: 3 ppm:  1365 
 p =  27.41% amt:    740000 sats  hops: 3 ppm:  1381 
 p =  37.09% amt:   2000000 sats  hops: 4 ppm:  1376 
 p =  15.19% amt:   1000000 sats  hops: 4 ppm:  1386 
 p =  10.66% amt:   1000000 sat

In [7]:

payment_session = SyncSimulatedPaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False,
                                 use_negative_circle_solver=True)
#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=100,base=0,)

Round number:  1
Try to deliver 35697754 satoshi:
Time difference for read = 12ms
Maximum flow 35697754
Time difference for max flow = 6ms
Time difference for edges_with_flow flow info = 0ms
Total cost before optimizations: 62716.1
Time difference for setup early terminate = 5ms
Total cost after optimizations: 0.137288%, p=2.32831e-08%
cost after 0 rounds: 0.175686%
cost after 100: 0.137362%, p=2.32831e-08%
cost after 200: 0%, p=0%
cost after 400: 0%, p=0%
Time difference for total time = 133ms
190 rounds, satoshis=35697754
190 rounds

Statistics about 42 candidate onions:

successful attempts:
--------------------
 p =  71.99% amt:    239408 sats  hops: 2 ppm:  1338
 p =  82.17% amt:    223941 sats  hops: 2 ppm:  1381
 p =  95.97% amt:     32426 sats  hops: 2 ppm:  1381
 p = 100.00% amt:         3 sats  hops: 2 ppm:  1333
 p =  88.16% amt:     69897 sats  hops: 3 ppm:  1362
 p =  83.85% amt:    142063 sats  hops: 3 ppm:  1380
 p =  34.53% amt:   2586665 sats  hops: 3 ppm:  1351
 p =  

In [ ]:
payment_session = SyncSimulatedPaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False,
                                 use_negative_circle_solver=True,
                                 min_cost_binary="../target/release/min_cost_rs")
#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=100,base=0,)